# 2.3.1 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought

Auflösung ist ein zentraler Faktor für die OCR Qualität. Entsprechend hilft es, bei älteren Plänen den schriftlichen Teil spaltenweise als Bild abzuspeichern (Spalte = PDF-Seite). Maximale Auflösung => Fehlerquote minimieren. (siehe 2.1.1 bzgl. Nutzungsschablone)

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
jarvis = OpenAI(instructions)
parser = Parser()
runner = Runner()

### 1) Einspaltiges Layout

* img_prompts

In [23]:
# 1) Textteil – ONE COL WITH CONTEXT
%store -r msgbaunvo
context_all = parser.text2prompts([msgbaunvo])
tt_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-TT-crop.png"
# ze_tt_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop.png"

instruction = 'Extrahiere alle planungsrechtlichen Festsetzungen'
async def run():
    return await jarvis.extractTextFromImagesWithContexts(instruction, [tt_crop_path], [context_all])
    
await runner.async_consistency_check(run)

['### Planungsrechtliche Festsetzungen (§ 9 (1) BBauG u. BauNVO)

#### 1.1 Bauliche Nutzung

- **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**
  - Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.

- **1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.**

- **1.1.3 Zahl der Vollgeschosse entsprechend den Einschrieben im Plan (§ 18 BauNVO und § 2 (4) LBO).**

#### 1.2 Bauweise (§ 22 BauNVO, siehe Einschriebe im Plan)

#### 1.3 Stellung der Gebäude (§ 9 (1) 1 b BBauG) und Firsthöhe wie im Plan eingezeichnet.

#### 1.4 Nebenanlagen im Sinne des § 14 BauNVO sind in den nicht überbaubaren Grundstücksflächen nicht zugelassen. Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen. Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, einem gedeckten Ein

[['### Planungsrechtliche Festsetzungen (§ 9 (1) BBauG u. BauNVO)\n\n#### 1.1 Bauliche Nutzung\n\n- **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**\n  - Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n\n- **1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.**\n\n- **1.1.3 Zahl der Vollgeschosse entsprechend den Einschrieben im Plan (§ 18 BauNVO und § 2 (4) LBO).**\n\n#### 1.2 Bauweise (§ 22 BauNVO, siehe Einschriebe im Plan)\n\n#### 1.3 Stellung der Gebäude (§ 9 (1) 1 b BBauG) und Firsthöhe wie im Plan eingezeichnet.\n\n#### 1.4 Nebenanlagen im Sinne des § 14 BauNVO sind in den nicht überbaubaren Grundstücksflächen nicht zugelassen. Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen. Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, ein

In [2]:
# 1) Textteil – ONE COL WITH CONTEXT
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
async def run():
    return await jarvis.extractTextFromImagesWithContexts(instruction, [tt_crop_path], [context_all])
    
await runner.async_consistency_check(run)

['### Art der baulichen Nutzung

1.1 **Bauliche Nutzung**
   - **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**
     - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
   - **1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO**
     - Gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.
   - **1.1.3 Zahl der Vollgeschosse**
     - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

### Maß der baulichen Nutzung

1.1 **Bauliche Nutzung**
   - **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**
     - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
   - **1.1.3 Zahl der Vollgeschosse**
     - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

### Bauweise

1.2 **Bauweise**
   - **§ 22 BauNVO**
     - Siehe Einzeichnungen im Plan.

### Überb

[['### Art der baulichen Nutzung\n\n1.1 **Bauliche Nutzung**\n   - **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**\n     - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n   - **1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO**\n     - Gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.\n   - **1.1.3 Zahl der Vollgeschosse**\n     - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).\n\n### Maß der baulichen Nutzung\n\n1.1 **Bauliche Nutzung**\n   - **1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO)**\n     - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n   - **1.1.3 Zahl der Vollgeschosse**\n     - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).\n\n### Bauweise\n\n1.2 **Bauweise**\n   - **§ 22 BauNVO**\n     - Siehe Einzeichnun

In [19]:
# 1.B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'
async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])
    
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan:

1.1 Bauliche Nutzung
1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).
    - Soweit es von Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.']
#############################################
['Die Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan sind wie folgt:

1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).
- Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.

1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.

1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan

[['Hier sind die extrahierten Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan:\n\n1.1 Bauliche Nutzung\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).\n    - Soweit es von Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.'],
 ['Die Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan sind wie folgt:\n\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).\n- Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n\n1.1.2 Ausnahmen im Sinne des § 3 (3) BauNVO sind gemäß § 1 (4) BauNVO nicht Bestandteil des Bebauungsplanes und daher nicht zulässig.\n\n1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

In [20]:
# 1.C3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'
async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])
    
await runner.async_consistency_check(run)

['Im Bebauungsplan sind folgende Festsetzungen zum Thema "Maß der baulichen Nutzung" enthalten:

1.1 Bauliche Nutzung
1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).
    - Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.
1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).']
#############################################
['Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan:

1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO):
- Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.

1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

2.1 Gebäudehöhen gemessen im Schnittpunkt zwischen Hauswand, neuem Gelände und Sparrenoberkante:
- 1. Vollgeschoss 3,50 m, für ein w

[['Im Bebauungsplan sind folgende Festsetzungen zum Thema "Maß der baulichen Nutzung" enthalten:\n\n1.1 Bauliche Nutzung\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO).\n    - Teilweise ist, wenn es vom Gelände und den Festsetzungen her möglich ist, ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).'],
 ['Hier sind die extrahierten Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan:\n\n1.1.1 Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO):\n- Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n\n1.1.3 Zahl der Vollgeschosse entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).\n\n2.1 Gebäudehöhen gemessen im Schnittpunkt zwischen Hauswand, neuem Gelände und Sparrenoberkante:\n- 1. Vollgeschoss 3,50 m, für ein weiteres Vollgeschoss 2,75 m.']

In [22]:
# 1.D3) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'
async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])
    
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan:

### Bauweise, überbaubare Grundstücksflächen

1. **Bauweise (§ 22 BauNVO)**
   - Siehe Einzeichnungen im Plan.

2. **Stellung der Gebäude (§ 9 (1) 1 b BBauG)**
   - Die Stellung der Gebäude sowie die Firstrichtung sind im Plan eingezeichnet.

3. **Nebenanlagen (§ 14 BauNVO)**
   - Nebenanlagen sind in den nicht überbaubaren Grundstücksflächen nicht zugelassen.
   - Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen.
   - Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, einem gedeckten Einstellplatz oder einer sonstigen baulichen Einbindung in die Gesamtanlage zulässig.

4. **Oberirdische Leitungen**
   - Oberirdische Leitungen sind nicht zulässig.

5. **Aufschüttungen und Abgrabungen**
   - Aufschüttungen und Abgrabungen sind auf ein Mindestmaß zu beschränken und dem Gelände anzupassen.
   - Sie sind im

[['Hier sind die extrahierten Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan:\n\n### Bauweise, überbaubare Grundstücksflächen\n\n1. **Bauweise (§ 22 BauNVO)**\n   - Siehe Einzeichnungen im Plan.\n\n2. **Stellung der Gebäude (§ 9 (1) 1 b BBauG)**\n   - Die Stellung der Gebäude sowie die Firstrichtung sind im Plan eingezeichnet.\n\n3. **Nebenanlagen (§ 14 BauNVO)**\n   - Nebenanlagen sind in den nicht überbaubaren Grundstücksflächen nicht zugelassen.\n   - Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen.\n   - Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, einem gedeckten Einstellplatz oder einer sonstigen baulichen Einbindung in die Gesamtanlage zulässig.\n\n4. **Oberirdische Leitungen**\n   - Oberirdische Leitungen sind nicht zulässig.\n\n5. **Aufschüttungen und Abgrabungen**\n   - Aufschüttungen und Abgrabungen sind auf ein Mindestmaß zu beschränken und dem Gelände anzupass

In [24]:
# 1) Textteil – ONE COL WITHOUT CONTEXT
tt_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-TT-crop.png"
# ze_tt_crop_path = "../data/processed/bpläne/1_alles_in_einem_dokument/L04-01-ZE-TT-bb-crop.png"

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksflächen.'

async def run():
    return await jarvis.extractTextFromImages(instruction, [tt_crop_path])
    
await runner.async_consistency_check(run)

['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan:

### Art der baulichen Nutzung
- **Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO):**
  - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.

### Maß der baulichen Nutzung
- **Zahl der Vollgeschosse:**
  - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).

### Bauweise
- **Bauweise (§ 22 BauNVO):**
  - Siehe Einzeichnungen im Plan.
- **Stellung der Gebäude (§ 9 (1) 1 b BBauG):**
  - Die Stellung der Gebäude sowie die Firstrichtung sind im Plan eingezeichnet.

### Überbaubare Grundstücksflächen
- **Nebenanlagen (§ 14 BauNVO):**
  - In den nicht überbaubaren Grundstücksflächen nicht zugelassen.
  - Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen.
  - Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, einem gedeckten Einstellpla

[['Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Bebauungsplan:\n\n### Art der baulichen Nutzung\n- **Art und Maß der baulichen Nutzung (§§ 16 - 21 BauNVO):**\n  - Soweit es vom Gelände und den Festsetzungen her möglich ist, ist ein weiteres Geschoss als anrechenbares Vollgeschoss zulässig.\n\n### Maß der baulichen Nutzung\n- **Zahl der Vollgeschosse:**\n  - Entsprechend den Einzeichnungen im Plan (§ 18 BauNVO und § 2 (4) LBO).\n\n### Bauweise\n- **Bauweise (§ 22 BauNVO):**\n  - Siehe Einzeichnungen im Plan.\n- **Stellung der Gebäude (§ 9 (1) 1 b BBauG):**\n  - Die Stellung der Gebäude sowie die Firstrichtung sind im Plan eingezeichnet.\n\n### Überbaubare Grundstücksflächen\n- **Nebenanlagen (§ 14 BauNVO):**\n  - In den nicht überbaubaren Grundstücksflächen nicht zugelassen.\n  - Garagen sind im Haus oder an den dafür ausgewiesenen Stellen mit einem Dach zu erstellen.\n  - Freistehende Einzelgaragen sind nur in Verbindung mit einer Sichtschutzwand, einem 

### 2) Zweispaltiges Layout

* img_prompts

In [10]:
# 2) Textteil – MULTI COL WITH CONTEXT
%store -r msgbaunvo
folder_path = "../data/processed/bpläne/1_alles_in_einem_dokument"
img_name = "/L22-01-TT-crop"
tt_prompts = parser.imageseries2prompts(folder_path, img_name)
context_all = parser.text2prompts([msgbaunvo])

def run():
    return jarvis.requestWithContext(context_all, [
        *tt_prompts,
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
            # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
        }
    ])
    
runner.consistency_check(run)

### Art der baulichen Nutzung:
1.1 Art der baulichen Nutzung (§§ 1-15 BauNVO)
1.1.1 Baugebiete (§ 1 Abs. 3 BauNVO)
1.1.1.1 Allgemeines Wohngebiet (§ 4 BauNVO)
- Zulässig sind:
  1. Wohngebäude
  2. nicht störende Handwerksbetriebe
  3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke

1.1.2 Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)
- Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:
  1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.

1.1.3 Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)
- Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.

### Maß der baulichen Nutzung:
1.2 Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)
- Z: I
- GRZ: 0,4
- GF: 0,5

### Bauweise, überbaubare Grundstücksfläche:
1.3 Bauweise (§ 22 BauNVO)
- E = offene Bauweise, nur Einzelhäuser zulässig
- a 

['### Art der baulichen Nutzung:\n1.1 Art der baulichen Nutzung (§§ 1-15 BauNVO)\n1.1.1 Baugebiete (§ 1 Abs. 3 BauNVO)\n1.1.1.1 Allgemeines Wohngebiet (§ 4 BauNVO)\n- Zulässig sind:\n  1. Wohngebäude\n  2. nicht störende Handwerksbetriebe\n  3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke\n\n1.1.2 Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)\n- Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:\n  1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.\n\n1.1.3 Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)\n- Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.\n\n### Maß der baulichen Nutzung:\n1.2 Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)\n- Z: I\n- GRZ: 0,4\n- GF: 0,5\n\n### Bauweise, überbaubare Grundstücksfläche:\n1.3 Bauweise (§ 22 BauNVO)\n- E = offene Bauweise, nur 

In [12]:
# 2.B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu dem Thema "Art der baulichen Nutzung".'
        }
    ])

runner.consistency_check(run)

Unter dem Thema "Art der baulichen Nutzung" sind im Bebauungsplan "Lindensteig I - Erweiterung" folgende Informationen festgelegt:

1. **Art der baulichen Nutzung (§§ 1-15 BauNVO)**

   1.1 **Baugebiete (§ 1 Abs. 3 BauNVO)**
   
   1.1.1 **Allgemeines Wohngebiet (§ 4 BauNVO)**
   
   Zulässig sind:
   1. Wohngebäude
   2. nicht störende Handwerksbetriebe
   3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke

   1.1.2 **Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)**
   
   Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:
   1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.

   1.1.3 **Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)**
   
   Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.
#############################################
Unter dem Thema "Art der baulichen Nutz

['Unter dem Thema "Art der baulichen Nutzung" sind im Bebauungsplan "Lindensteig I - Erweiterung" folgende Informationen festgelegt:\n\n1. **Art der baulichen Nutzung (§§ 1-15 BauNVO)**\n\n   1.1 **Baugebiete (§ 1 Abs. 3 BauNVO)**\n   \n   1.1.1 **Allgemeines Wohngebiet (§ 4 BauNVO)**\n   \n   Zulässig sind:\n   1. Wohngebäude\n   2. nicht störende Handwerksbetriebe\n   3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke\n\n   1.1.2 **Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)**\n   \n   Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:\n   1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.\n\n   1.1.3 **Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)**\n   \n   Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.',
 'Unter dem Thema "Art der baulichen Nutzung" sind im Bebauun

In [13]:
# 2.C3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu dem Thema "Maß der baulichen Nutzung".'
        }
    ])

runner.consistency_check(run)

Die Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Lindensteig I - Erweiterung" der Stadt Laichingen sind wie folgt:

### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)
- **Z**: 1
- **GRZ**: 0,4
- **GF**: 0,5
#############################################
Unter dem Thema "Maß der baulichen Nutzung" sind im Bebauungsplan "Lindensteig I - Erweiterung" folgende Informationen festgesetzt:

1.3 Bauweise (§ 22 BauNVO)
- E = offene Bauweise, nur Einzelhäuser zulässig
- a = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)

1.4 Garagen und Nebenanlagen (§§ 12 und 14 BauNVO)
- Garagen und Nebenanlagen sind nur innerhalb der Baugrenzen zulässig.
- Garagen sind mit dem Hauptgebäude zu verknüpfen oder in das Dach des Wohngebäudes einzubeziehen.
- Bei giebelständiger Grenzbebauung ist als abweichende Bauweise zulässig:
  - eine Gesamthöhe von 5,0 m
  - eine mittlere Gebäudehöhe von 4,0 m
  - eine Wandfläche, gemessen über fertigem Gelän

['Die Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan "Lindensteig I - Erweiterung" der Stadt Laichingen sind wie folgt:\n\n### Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)\n- **Z**: 1\n- **GRZ**: 0,4\n- **GF**: 0,5',
 'Unter dem Thema "Maß der baulichen Nutzung" sind im Bebauungsplan "Lindensteig I - Erweiterung" folgende Informationen festgesetzt:\n\n1.3 Bauweise (§ 22 BauNVO)\n- E = offene Bauweise, nur Einzelhäuser zulässig\n- a = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)\n\n1.4 Garagen und Nebenanlagen (§§ 12 und 14 BauNVO)\n- Garagen und Nebenanlagen sind nur innerhalb der Baugrenzen zulässig.\n- Garagen sind mit dem Hauptgebäude zu verknüpfen oder in das Dach des Wohngebäudes einzubeziehen.\n- Bei giebelständiger Grenzbebauung ist als abweichende Bauweise zulässig:\n  - eine Gesamthöhe von 5,0 m\n  - eine mittlere Gebäudehöhe von 4,0 m\n  - eine Wandfläche, gemessen über fertigem Gelände, von 28 m²\n\nAnsons

In [14]:
# 2.D3) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu dem Thema "Bauweise, überbaubare Grundstücksflächen".'
        }
    ])

runner.consistency_check(run)

Die Informationen zu dem Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan "Lindensteig I - Erweiterung" der Stadt Laichingen sind wie folgt:

### 1.3 Bauweise (§ 22 BauNVO)
- **E** = offene Bauweise, nur Einzelhäuser zulässig
- **a** = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)

### 1.4 Garagen und Nebenanlagen (§§ 12 und 14 BauNVO)
- Garagen und Nebenanlagen sind nur innerhalb der Baugrenzen zulässig.
- Garagen sind mit dem Hauptgebäude zu verknüpfen oder in das Dach des Wohngebäudes einzubeziehen.
- Bei giebelständiger Grenzbebauung ist als abweichende Bauweise zulässig:
  - eine Gesamthöhe von 5,0 m
  - eine mittlere Gebäudehöhe von 4,0 m
  - eine Wandfläche, gemessen über fertigem Gelände, von 28 m²
- Ansonsten gelten die Maße der LBO.

### 2.6 Genehmigungspflicht für Aufschüttungen und Abgrabungen (§ 73 Abs. 2 Nr. 1 LBO)
- Abweichend von § 52 Abs. 1 Nr. 17 LBO bedürfen Aufschüttungen und Abgrabungen über 0,8 m Höhenu

['Die Informationen zu dem Thema "Bauweise, überbaubare Grundstücksflächen" aus dem Bebauungsplan "Lindensteig I - Erweiterung" der Stadt Laichingen sind wie folgt:\n\n### 1.3 Bauweise (§ 22 BauNVO)\n- **E** = offene Bauweise, nur Einzelhäuser zulässig\n- **a** = abweichende Bauweise für Garagen, die giebelständig auf der Grenze stehen (siehe 1.4)\n\n### 1.4 Garagen und Nebenanlagen (§§ 12 und 14 BauNVO)\n- Garagen und Nebenanlagen sind nur innerhalb der Baugrenzen zulässig.\n- Garagen sind mit dem Hauptgebäude zu verknüpfen oder in das Dach des Wohngebäudes einzubeziehen.\n- Bei giebelständiger Grenzbebauung ist als abweichende Bauweise zulässig:\n  - eine Gesamthöhe von 5,0 m\n  - eine mittlere Gebäudehöhe von 4,0 m\n  - eine Wandfläche, gemessen über fertigem Gelände, von 28 m²\n- Ansonsten gelten die Maße der LBO.\n\n### 2.6 Genehmigungspflicht für Aufschüttungen und Abgrabungen (§ 73 Abs. 2 Nr. 1 LBO)\n- Abweichend von § 52 Abs. 1 Nr. 17 LBO bedürfen Aufschüttungen und Abgrabungen

In [14]:
# 2) Textteil – MULTI COL WITHOUT CONTEXT
folder_path = "../data/processed/bpläne/1_alles_in_einem_dokument"
img_name = "/L22-01-TT-crop"
tt_prompts = parser.imageseries2prompts(folder_path, img_name)

def run():
    return jarvis.request([
        *tt_prompts,
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
            # Optional: Zusätzlich Dach, Garage, Stelltung und Stellplatz
        }
    ])
    
runner.consistency_check(run)


### Art der baulichen Nutzung:
1.1 Art der baulichen Nutzung (§§ 1-15 BauNVO)
1.1.1 Baugebiete (§ 1 Abs. 3 BauNVO)
1.1.1.1 Allgemeines Wohngebiet (§ 4 BauNVO)
Zulässig sind:
1. Wohngebäude
2. nicht störende Handwerksbetriebe
3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke

1.1.2 Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)
Von den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:
1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.

1.1.3 Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)
Ausnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.

### Maß der baulichen Nutzung:
1.2 Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)
Z: 1
GRZ: 0,4
GF: 0,5

### Bauweise:
1.3 Bauweise (§ 22 BauNVO)
E = offene Bauweise, nur Einzelhäuser zulässig
a = abweichende Bauweise für Garagen, die giebelständig a

['### Art der baulichen Nutzung:\n1.1 Art der baulichen Nutzung (§§ 1-15 BauNVO)\n1.1.1 Baugebiete (§ 1 Abs. 3 BauNVO)\n1.1.1.1 Allgemeines Wohngebiet (§ 4 BauNVO)\nZulässig sind:\n1. Wohngebäude\n2. nicht störende Handwerksbetriebe\n3. Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke\n\n1.1.2 Ausschluss bestimmter Arten von allgemein zulässigen Nutzungen (§ 1 Abs. 5 und 9 BauNVO)\nVon den Nutzungen im Sinne des § 4 Abs. 2 Nr. 2 BauNVO sind folgende bestimmte Arten bzw. bauliche oder sonstige Anlagen nicht zulässig:\n1. Die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften.\n\n1.1.3 Zulässigkeit von Ausnahmen (§ 1 Abs. 6 BauNVO)\nAusnahmen im Sinne des § 4 Abs. 3 Nr. 1 bis 5 sind nicht zulässig.\n\n### Maß der baulichen Nutzung:\n1.2 Maß der baulichen Nutzung (§§ 16 - 21a BauNVO)\nZ: 1\nGRZ: 0,4\nGF: 0,5\n\n### Bauweise:\n1.3 Bauweise (§ 22 BauNVO)\nE = offene Bauweise, nur Einzelhäuser zulässig\na = abweichende Bauweise für G